# Level 2 get dataframes from bigquery public-data:google_analytics_sample

Getting concurrent request to google big data API and create dataframe

In [262]:
import concurrent.futures
import os

from google.cloud import bigquery
import pandas as pd

creds = "level2_key.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds

client = bigquery.Client()


def fetch_data(limit):
    offset = limit
    sql_query = f"""
    SELECT visitNumber, visitId, visitStartTime, date, totals.timeOnSite, totals.hits, totals.pageviews 
    trafficSource, device.browser, geoNetwork.subContinent, geoNetwork.country, fullVisitorId, channelGrouping
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
    LIMIT 1000 OFFSET {offset}
    """
    return client.query(sql_query).to_dataframe()


limits = [0, 1000, 2000]

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_data, limit) for limit in limits]

    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if 'df_gns' not in locals():
            df_gns = result
        else:
            df_gns = pd.concat([df_gns, result], ignore_index=True)

df_gns

,visitNumber,visitId,visitStartTime,date,timeOnSite,hits,trafficSource,browser,subContinent,country,fullVisitorId,channelGrouping
0,1,1501619106,1501619106,20170801,<NA>,1,1,Safari,Western Europe,Netherlands,6229020639379611564,Organic Search
1,1,1501638962,1501638962,20170801,<NA>,1,1,Safari,Northern America,United States,4702728182802231974,Direct
2,3,1501598053,1501598053,20170801,<NA>,1,1,Safari,Northern America,United States,5964000044560393406,Direct
3,1,1501596233,1501596233,20170801,<NA>,1,1,Safari,Northern America,United States,171086260572741738,Organic Search
4,1,1501598282,1501598282,20170801,<NA>,1,1,Chrome,Northern America,United States,1964635174785924193,Organic Search
...,...,...,...,...,...,...,...,...,...,...,...,...
7663,1,1501603556,1501603556,20170801,83,5,5,Chrome,Northern America,United States,4660658311737191205,Direct
7664,7,1501634127,1501634127,20170801,98,5,5,Chrome,Northern America,United States,5199370466032130686,Paid Search
7665,4,1501624586,1501624586,20170801,85,5,5,Chrome,Northern America,United States,3824960045676496699,Direct
7666,3,1501606850,1501606850,20170801,778,5,3,Chrome,Northern America,United States,1580431283728289185,Direct


In [217]:
time_on_site_in_country = df_gns.groupby("country")["timeOnSite"].sum()
popular_browsers_in_countries = df_gns.groupby("country")["browser"].max()
popular_channel_in_country = df_gns.groupby("country")["channelGrouping"].max()

count_result = df_gns.groupby(["country", "channelGrouping"]).size().reset_index(name="count")
count_popular_channel_in_country = count_result.groupby("country")["count"].max()

country_df = pd.DataFrame({
    "totalTimeOnSite": time_on_site_in_country,
    "popularBrowser": popular_browsers_in_countries,
    "popularChannel": popular_channel_in_country,
    "countPopularChannel": count_popular_channel_in_country
})
country_df

,totalTimeOnSite,popularBrowser,popularChannel,countPopularChannel
country,,,,
(not set),92,Safari,Social,4
Algeria,537,Opera,Social,4
Argentina,0,Firefox,Social,5
Australia,6245,Safari,Social,33
Austria,426,Internet Explorer,Organic Search,6
...,...,...,...,...
United Arab Emirates,1700,Safari,Social,5
United Kingdom,15098,Safari (in-app),Social,101
United States,302259,Safari (in-app),Social,571


In [238]:
average_time_on_site_on_browser = df_gns.groupby("browser")["timeOnSite"].mean()
count_using_browser_for_channels = df_gns.groupby("browser")["channelGrouping"].count()
popular_channel_on_browser = df_gns.groupby("browser")["channelGrouping"].max()

count_browser_result = df_gns.groupby(["browser", "channelGrouping"]).size().reset_index(name="count")
count_popular_channel_on_browser = count_browser_result.groupby("browser")["count"].max()

count_visits_from_browser = df_gns.groupby("browser")["visitNumber"].count()

browser_df = pd.DataFrame({
    "averageTimeOnBrowser": average_time_on_site_on_browser,
    "count_use_channels": count_using_browser_for_channels,
    "popularChannel": popular_channel_on_browser,
    "countPopularChannel": count_popular_channel_on_browser,
    "visits": count_visits_from_browser
})
browser_df

,averageTimeOnBrowser,count_use_channels,popularChannel,countPopularChannel,visits
browser,,,,,
Android Browser,36.0,4,Referral,2,4
Android Webview,223.090909,38,Social,10,38
Chrome,346.37793,3800,Social,1866,3800
Coc Coc,<NA>,4,Social,2,4
Edge,238.454545,46,Social,22,46
Firefox,266.475,202,Social,112,202
Internet Explorer,310.5,108,Social,40,108
Mozilla Compatible Agent,<NA>,2,Social,2,2
Nokia Browser,177.0,4,Referral,2,4


In [242]:
time_on_channel = df_gns.groupby("channelGrouping")["timeOnSite"].count()
popular_browsers_for_channel = df_gns.groupby("channelGrouping")["browser"].max()

count_channel_result = df_gns.groupby(["channelGrouping", "browser"]).size().reset_index(name="count")
count_popular_channel_on_browser = count_channel_result.groupby("channelGrouping")["count"].max()

channel_df = pd.DataFrame({
    "timeOnChannel": time_on_channel,
    "countBrowserForChannel": count_popular_channel_on_browser,
    "popularBrowser": popular_browsers_for_channel,
})
channel_df

,timeOnChannel,countBrowserForChannel,popularBrowser
channelGrouping,,,
Affiliates,44,78,Safari
Direct,402,594,Safari (in-app)
Display,54,66,Safari
Organic Search,1254,1866,UC Browser
Paid Search,96,112,Safari (in-app)
Referral,654,838,YaBrowser
Social,116,246,YaBrowser


In [261]:
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from concurrent.futures import ThreadPoolExecutor

def connection(worksheet: str) -> gspread:
    credential = ServiceAccountCredentials.from_json_keyfile_name("keys.json")
    file = gspread.authorize(credential)
    workbook = file.open("BigQuery")
    sheet = workbook.worksheet(worksheet)
    return sheet

def write_to_google_sheet(df: pd.DataFrame, worksheet: str) -> None:
    sheet = connection(worksheet)
    sheet.clear()
    set_with_dataframe(sheet, df, include_index=True)

tasks = [
    (country_df, "Country"),
    (browser_df, "Browser"),
    (channel_df, "Channel")
]

def write_concurrently(task):
    write_to_google_sheet(task[0], task[1])

with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(write_concurrently, tasks)